In [ ]:
!pip install folium
!pip install geocoder
!pip install tqdm

In [1]:
import pandas as pd
import numpy as np
import geocoder
import folium
from tqdm import tqdm
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Carregando banco de dados.

In [2]:
# Import pandas
import pandas as pd

# Assign spreadsheet filename: file
file = 'imd_student_blind.xlsx'

# Load spreadsheet: xl
xl = pd.ExcelFile(file)

# Print sheet names
print(xl.sheet_names)

# Load a sheet into a DataFrame by index: df
df = xl.parse(0)

# Print the head of the DataFrame df
df.head()

['Sheet1']


,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado


In [3]:
# Load a sheet into a DataFrame by index: df
addr = pd.ExcelFile('imd_student_blind_coord.xlsx').parse(0)

# Print the head of the DataFrame df
addr.head()

,CEP,logradouro,bairro,cidade,uf,lat,long
0,0,NaN,NaN,NaN,NaN,0.000000,0.000000
1,59064320,Rua João Celso Filho,Lagoa Nova,Natal,RN,-5.827858,-35.223075
2,59064330,Rua Anísio de Souza,Candelária,Natal,RN,-5.828610,-35.222589
3,59056140,Rua Dom Joaquim de Almeida,Lagoa Nova,Natal,RN,-5.815763,-35.201021
4,59066380,Rua Comandante Monteiro Chaves,Pitimbu,Natal,RN,-5.865686,-35.223363


## Localização por CEP.

In [4]:
# Localização dos estudantes
from folium.plugins import MarkerCluster

# Create map object
mapa = folium.Map(location=[-5, -35], zoom_start = 2, tiles = 'Cartodb Positron')

marker_cluster = folium.MarkerCluster().add_to(mapa)

for i in tqdm(range(len(addr))):
    if addr.ix[i,'lat'] != '' and addr.ix[i,'lat'] != 0 and addr.ix[i,'long'] != '' and addr.ix[i,'long'] != 0:
        folium.Marker(
            location=[addr.ix[i,'lat'], addr.ix[i,'long']],
            popup= str(addr.ix[i,'bairro']) + ', ' + str(addr.ix[i,'cidade']) + ', ' + str(addr.ix[i,'uf']),
            icon=folium.Icon(color='green', icon='info-sign')).add_to(marker_cluster)

mapa

100%|████████████████████████████████████████████████████████████████████████████████| 627/627 [00:29<00:00, 21.88it/s]


## Médias por CEP

In [5]:
# HeatMap por notas
#from folium.plugins import HeatMap
#
#coordinates = []
#
#for i in tqdm(range(len(df))):
#    location = addr.loc[addr['CEP'] == df.ix[i,'CEP']]
#    if all(~np.isnan([location.lat.item(), location.long.item(), df.ix[i,'nota']])):
#        if location.lat.item() != '' and location.lat.item() != 0 and location.long.item() != '' and location.long.item() != 0:
#             coordinates.append([location.lat.item(), location.long.item(), df.ix[i,'nota']])
# 
#
# Create map object
#m = folium.Map(location=[-5, -35], zoom_start = 3, tiles = 'Cartodb Positron')
#
#HeatMap(coordinates).add_to(m)
#m

In [6]:
# HeatMap Médias por CEP
from folium.plugins import HeatMap

coordinates = []

for i in tqdm(range(len(addr))):
    mg = df['nota'].median()
    d = df.loc[df['CEP'] == addr.ix[i,'CEP']]
    dist = abs(d['nota'] - mg)
    m = addr.ix[i,'CEP']
    
    if all(~np.isnan([addr.ix[i,'lat'], addr.ix[i,'long'], m])):
        if addr.ix[i,'lat'] != '' and addr.ix[i,'lat'] != 0 and addr.ix[i,'long'] != '' and addr.ix[i,'long'] != 0:
             coordinates.append([addr.ix[i,'lat'], addr.ix[i,'long'], m])
 
# Create map object
m = folium.Map(location=[-5, -35], zoom_start = 3, tiles = 'Cartodb Positron')

HeatMap(coordinates).add_to(m)

m

100%|███████████████████████████████████████████████████████████████████████████████| 627/627 [00:01<00:00, 472.03it/s]


## Localização - Cancelamento de curso por ano

In [7]:
# Localização por cancelamento de curso 2014
from folium.plugins import MarkerCluster

# Create map object
mapa = folium.Map(location=[-5, -35], zoom_start = 4, tiles = 'Cartodb Positron')

marker_cluster = folium.MarkerCluster().add_to(mapa)

for i in tqdm(range(len(df))):
    location = addr.loc[addr['CEP'] == df.ix[i,'CEP']]
    if location.lat.item() != '' and location.lat.item() != 0 and location.long.item() != '' and location.long.item() != 0:
        if (df.ix[i,'status'] == 'CANCELADO' and df.ix[i,'ano_disciplina'] == 2014):
            folium.Marker(
                location=[location.lat.item(), location.long.item()],
                popup= str(location.bairro.item()) + ', ' + str(location.cidade.item()) + ', ' + str(location.uf.item()),
                icon=folium.Icon(color='green', icon='info-sign')).add_to(marker_cluster)
        
mapa

100%|█████████████████████████████████████████████████████████████████████████████| 4842/4842 [00:22<00:00, 219.51it/s]


In [8]:
# Localização por cancelamento de curso 2015
from folium.plugins import MarkerCluster

# Create map object
mapa = folium.Map(location=[-5, -35], zoom_start = 4, tiles = 'Cartodb Positron')

marker_cluster = folium.MarkerCluster().add_to(mapa)

for i in tqdm(range(len(df))):
    location = addr.loc[addr['CEP'] == df.ix[i,'CEP']]
    if location.lat.item() != '' and location.lat.item() != 0 and location.long.item() != '' and location.long.item() != 0:
        if (df.ix[i,'status'] == 'CANCELADO' and df.ix[i,'ano_disciplina'] == 2015):
            folium.Marker(
                location=[location.lat.item(), location.long.item()],
                popup= str(location.bairro.item()) + ', ' + str(location.cidade.item()) + ', ' + str(location.uf.item()),
                icon=folium.Icon(color='green', icon='info-sign')).add_to(marker_cluster)
        
mapa

100%|█████████████████████████████████████████████████████████████████████████████| 4842/4842 [00:21<00:00, 227.85it/s]


In [9]:
# Localização por cancelamento de curso 2014
from folium.plugins import MarkerCluster

# Create map object
mapa = folium.Map(location=[-5, -35], zoom_start = 4, tiles = 'Cartodb Positron')

marker_cluster = folium.MarkerCluster().add_to(mapa)

for i in tqdm(range(len(df))):
    location = addr.loc[addr['CEP'] == df.ix[i,'CEP']]
    if location.lat.item() != '' and location.lat.item() != 0 and location.long.item() != '' and location.long.item() != 0:
        if (df.ix[i,'status'] == 'CANCELADO' and df.ix[i,'ano_disciplina'] == 2016):
            folium.Marker(
                location=[location.lat.item(), location.long.item()],
                popup= str(location.bairro.item()) + ', ' + str(location.cidade.item()) + ', ' + str(location.uf.item()),
                icon=folium.Icon(color='green', icon='info-sign')).add_to(marker_cluster)
        
mapa

100%|█████████████████████████████████████████████████████████████████████████████| 4842/4842 [00:15<00:00, 312.10it/s]
